In [93]:
import pandas as pd

In [94]:
df = pd.read_excel("DENEMEVERİLER.xlsx") # veri dosyasını okuyoruz

In [95]:
df = df.drop_duplicates(keep='first')   # Kopya verilerin tekini bırakıp tekini siliyoruz

In [96]:
df["Fiyat"] = df["Fiyat"].str.replace(" TL", "", regex=False).str.replace(".", "", regex=False).astype("float64")

In [97]:
def temizle_m2(deger):
    try:
        return float(str(deger).replace(" m²", "").replace(",", "."))
    except:
        return None

df["Net m2"] = df["Net m2"].apply(temizle_m2)
df["Brüt m2"] = df["Brüt m2"].apply(temizle_m2)


In [98]:
def ayir_oda_salon(deger):
    try:
        oda, salon = deger.split("+")
        return pd.Series([int(oda), int(salon)])
    except:
        return pd.Series([9, 1])    # Bazı değerler 9+Oda şeklinde yazılmış o değerlere atama

df[["Oda", "Salon"]] = df["Oda Sayısı"].apply(ayir_oda_salon)

In [99]:
def temizle_bina_yasi(yas):
    if pd.isna(yas):
        return None
    if "Yeni" in yas:
        return 0
    if "-" in yas:  # örn: 5-10
        min_yas, max_yas = yas.split("-")
        return (int(min_yas) + int(max_yas)) / 2
    if "Ve Üzeri" in yas:
        return 21
    try:
        return int(yas)
    except:
        return None

df["Bina Yaşı"] = df["Bina Yaşı"].apply(temizle_bina_yasi)

In [100]:
# 1. "Toplam Kat" içinde ısıtma tipi olanları tespit et
isitma_tipleri = ["Kombi Doğalgaz", "Yerden Isıtma", "Doğalgaz Sobalı", "Merkezi Doğalgaz", "Sobalı","Isıtma Yok","Klimalı","Kat Kaloriferi","Merkezi (Pay Ölçer)","Isı Pompası"]
mask_isitma_in_toplamkat = df["Toplam Kat"].astype(str).isin(isitma_tipleri)

In [101]:
# 2. Bu hatalı değerleri "Isıtma Tipi" sütununa taşı
df["Isıtma Tipi"] = df["Isıtma Tipi"].astype("object")   # Bu satır olmasa da olur ama ileride problem çıkma ihtimaline karşı pandas uyarı verdi
df.loc[mask_isitma_in_toplamkat, "Isıtma Tipi"] = df.loc[mask_isitma_in_toplamkat, "Toplam Kat"]

In [102]:
df["Toplam Kat"] = pd.to_numeric(df["Toplam Kat"], errors="coerce")


In [103]:
def temizle_kat(kat_str, toplam_kat):
    if pd.isna(kat_str):
        return 0
    kat_str = kat_str.lower()
    if "bahçe" in kat_str or "giriş" in kat_str or "zemin" in kat_str:
        return 0
    elif "çatı" in kat_str or "dubleks" in kat_str:
        return toplam_kat
    else:
        try:
            return float(kat_str.split(".")[0])
        except:
            return None

df["Kat (Sayı)"] = df.apply(lambda row: temizle_kat(row["Kat"], row["Toplam Kat"]), axis=1)

In [104]:
print(df.info)   # Veri seti hakkında genel bilgi (sütun sayısı, veri tipi, eksik değer var mı vs.)

print(df.dtypes)   # Her sütunun veri tipini gösterir – int, float, object gibi

print(df.describe())  # Sayısal sütunlar için istatistiksel özet – ortalama, std, min, max gibi değerleri verir

print(df.isnull().sum())  # Hangi sütunda kaç adet eksik (null/NaN) veri olduğunu listeler

<bound method DataFrame.info of            Fiyat  Net m2  Brüt m2 Oda Sayısı  Bina Yaşı           Kat  \
0      7200000.0   145.0    160.0        3+1        2.0         7.Kat   
1      4100000.0   115.0    135.0        2+1        0.0         6.Kat   
2      7800000.0   240.0    260.0        6+1       13.0         4.Kat   
3     19000000.0   202.0    249.0        4+1        7.5        17.Kat   
4      2300000.0   400.0    500.0        3+1       21.0         3.Kat   
...          ...     ...      ...        ...        ...           ...   
6310   3570000.0   115.0    297.0        4+0       21.0             1   
6311   2450000.0    95.0    110.0        2+1        0.0         1.Kat   
6312   1700000.0   120.0    234.0      4.5+1       18.0             1   
6313   1350000.0   100.0    124.0        4+3       21.0             2   
6314   2800000.0   140.0    160.0        3+1       13.0  Çatı Dubleks   

      Toplam Kat     Isıtma Tipi     İl       İlçe     Mahalle  \
0            7.0         

In [105]:
# Kat (Sayı) sütununu sayıya çevir
df["Kat (Sayı)"] = pd.to_numeric(df["Kat (Sayı)"], errors="coerce")

# Eksik değerleri mod ile doldur
df["Kat (Sayı)"] = df["Kat (Sayı)"].fillna(df["Kat (Sayı)"].mode()[0])
df["Toplam Kat"] = df.apply(
    lambda row: row["Kat (Sayı)"] + 1 if pd.isna(row["Toplam Kat"]) else row["Toplam Kat"],   # Boşlukları Kat sayısının 1 fazlasını alarak dolduruyorum
    axis=1
)

In [106]:
# "Toplam Kat" >= 4 olan ve "Site içinde mi" değeri "Hayır" olan satırları seçiyoruz
mask = (df["Toplam Kat"] >= 4) & (df["Site İçinde mi?"] == "Hayır")
hayir_index = df[mask].index

# Rastgele karıştırmak için sample kullanalım
hayir_index_shuffled = hayir_index.to_series().sample(frac=1, random_state=42)

# Güncellenecek miktarları hesapla(Yüzdelik hesabına göre yaptım %28.99 site içerisinde)
total = len(hayir_index_shuffled)
evet_count = int(total * 0.2899)
hayir_count = total - evet_count

# Güncellemeyi yap
df.loc[hayir_index_shuffled[:evet_count], "Site İçinde mi?"] = "Evet"
df.loc[hayir_index_shuffled[evet_count:], "Site İçinde mi?"] = "Hayır"

In [107]:
# Eksik Isıtma Tipi olanların indekslerini al
isitma_nan_index = df[df["Isıtma Tipi"].isna()].index

# Karışık sıralı rastgele indeksler oluştur 
shuffled_index = isitma_nan_index.to_series().sample(frac=1, random_state=42).values

# Toplam eksik sayısı
total = len(shuffled_index)

# Her kategorinin sayısı
kombi_count = int(total * 0.6303)
merkezi_count = int(total * 0.22)           # Sitedeki oranlara göre ayarladım
dogalgaz_yok_count = total - kombi_count - merkezi_count

# Rastgele karışık indeksleri kullanarak değerleri atıyoruz
df.loc[shuffled_index[:kombi_count], "Isıtma Tipi"] = "Kombi Doğalgaz"
df.loc[shuffled_index[kombi_count:kombi_count + merkezi_count], "Isıtma Tipi"] = "Merkezi Doğalgaz"
df.loc[shuffled_index[kombi_count + merkezi_count:], "Isıtma Tipi"] = "Isıtma Yok"

In [108]:
# tekrar kontrol ediyoruz 
print(df.isnull().sum()) 

# Tamamen aynı olan satırları getirir (her biri dahil)
duplicate_rows = df[df.duplicated(keep=False)]

# Kopya kayıtları görüntüle
print(duplicate_rows)

Fiyat              0
Net m2             0
Brüt m2            3
Oda Sayısı         0
Bina Yaşı          1
Kat                0
Toplam Kat         0
Isıtma Tipi        0
İl                 0
İlçe               0
Mahalle            0
Site İçinde mi?    0
Oda                0
Salon              0
Kat (Sayı)         0
dtype: int64
Empty DataFrame
Columns: [Fiyat, Net m2, Brüt m2, Oda Sayısı, Bina Yaşı, Kat, Toplam Kat, Isıtma Tipi, İl, İlçe, Mahalle, Site İçinde mi?, Oda, Salon, Kat (Sayı)]
Index: []


In [109]:
df_cleaned = df.drop(columns=["Kat", "Oda Sayısı"])   # Gereksiz sutünları kaldırıyoruz

In [110]:
df_cleaned.to_excel("temizVeri_Bursa.xlsx", index=False, engine="openpyxl")
print("✅ Temiz veri 'temizVeri_Bursa.xlsx' dosyasına kaydedildi.")

✅ Temiz veri 'temizVeri_Bursa.xlsx' dosyasına kaydedildi.
